In [1]:
import csv
from toolsdsgen import *

In [2]:
class Cameras:
  def __init__(self, cam_poses, focl, w, h):
    self.cam_poses = cam_poses
    self.focl = focl
    self.w = w
    self.h = h
    
def gen_data(cams, μ_w, p):
    
    e_c = (np.random.randint(0,2,(3,2))-0.5)*2*p # 3 cameras (x,y)
    e_s = np.random.normal(0,abs(p/4),(3,2)) # 3 cameras (x,y)    
    error_p = np.round(e_c + e_s, 0) # 3 cameras (x,y)    
    t_r2w, t_w2r = get_transmats(cams.cam_poses)
    r_r2w, r_w2r = get_rotmats(cams.cam_poses)


    aux_a = np.zeros((4,3))
    aux_b = np.zeros((3,3))

    μ_r = np.zeros((3,3)) 
    μ_f = np.zeros((4,4)) 

    cam_ang_fp = np.zeros((2,3))
    px = np.zeros(3)
    py = np.zeros(3)
    snn_x = np.zeros(3)
    snn_y = np.zeros(3) 

    # For each camera ... 
    for k in range(3):


        aux_a[:,k] = t_w2r[:,:, k] @ np.concatenate((μ_w,[1]))
        μ_r[:,k]  = r_w2r[:,:,k] @ aux_a[0:3,k]

        # Going Back to World Space from Real-Camera Space
        aux_b[:,k] = r_r2w[:,:,k] @ μ_r[0:3,k]
        μ_f[:,k] = t_r2w[:,:, k] @ np.concatenate((aux_b[:,k],[1]))

        # Estimating Which pixels are concerned
        cam_ang_fp[:,k] = get_angles_from_pos(μ_r[:,k])
        px[k], py[k] = get_dvs_from_angles(cam_ang_fp[:,k], cams.focl, k)
        snn_x[k] = (px[k]+error_p[k,0])/(cams.w/2)
        snn_y[k] = (py[k]+error_p[k,1])/(cams.h/2)
    
    
    return snn_x, snn_y
    

In [3]:
def get_datasets(cams, px_e, nb_copies, nb_samples, nb_pts_dim):
    
    # Creating linear spaces
    p_array = px_e*np.ones(nb_samples)
    x_array = np.linspace(-1.0,0.0,nb_pts_dim)
    y_array = np.linspace(0.0,1.0,nb_pts_dim)
    z_array = np.linspace(0.5,1.5,nb_pts_dim)

    with open(f"datasets/dataset_validation.csv", 'w', encoding='UTF8', newline='') as f_vl:
        writer_vl = csv.writer(f_vl)
        count = 0
        for c_ix in range(1,nb_copies+1):
            with open(f"datasets/dataset_noise_{px_e}px_{c_ix}.csv", 'w', encoding='UTF8', newline='') as f_tr:
                writer_tr = csv.writer(f_tr)
                for x in x_array:
                    for y in y_array:
                        for z in z_array:
                            μ_w = np.array([x, y, z])
                            snn_x_vl, snn_y_vl = gen_data(cams, μ_w, 0)
                            for p in p_array:                                
                                if abs(snn_x_vl[0])<=1 and abs(snn_x_vl[1])<=1 and abs(snn_x_vl[2])<=1:
                                    if abs(snn_y_vl[0])<=1 and abs(snn_y_vl[1])<=1 and abs(snn_y_vl[2])<=1:                                        
                                        snn_x_tr, snn_y_tr = gen_data(cams, μ_w, p)
                                        line_vl = np.round([x,y,z,snn_x_vl[0],snn_y_vl[0],snn_x_vl[1],snn_y_vl[1],snn_x_vl[2],snn_y_vl[2]],3)
                                        writer_vl.writerow(line_vl)
                                        line_tr = np.round([x,y,z,snn_x_tr[0],snn_y_tr[0],snn_x_tr[1],snn_y_tr[1],snn_x_tr[2],snn_y_tr[2]],3)
                                        writer_tr.writerow(line_tr)        
                                        count += 1

In [4]:
cam_poses = set_cam_poses()
focl = set_focal_lengths()
cams = Cameras(cam_poses, focl, 640, 480)

nb_copies = 20
nb_pts_dim = 16
nb_samples = 64
px_e_array = [0,4,8,12,16,20,24,32]


In [5]:
for px_e in px_e_array:
    get_datasets(cams, px_e, nb_copies, nb_samples, nb_pts_dim)